### 聚合方法
##### aggregate:
    返回一个字典。这是queryset的一个终端子句，从返回值为字典可知，其不能进一步执行过滤或查询操作。aggregate是对多个query对象中的某一个值进行计算汇总，得到一个值。
##### annotate：
     返回的是一个queryset，因此可以进一步执行查询或过滤操作。在一对多或多对多的关系中，一个模型对象关联着多个对象，倘若需要对关联的对象某些信息进行汇总，则可以使用annotate，其相当于为每一个query对象都添加一个动态属性，用于记录该汇总值。
    


In [ ]:
from django.db import models

class Author(models.Model):
    name = models.CharField(max_length=100)
    age = models.IntegerField()

class Publisher(models.Model):
    name = models.CharField(max_length=300)

class Book(models.Model):
    name = models.CharField(max_length=300)
    pages = models.IntegerField()
    price = models.DecimalField(max_digits=10, decimal_places=2)
    rating = models.FloatField()
    authors = models.ManyToManyField(Author)
    publisher = models.ForeignKey(Publisher, on_delete=models.CASCADE)
    pubdate = models.DateField()

class Store(models.Model):
    name = models.CharField(max_length=300)
    books = models.ManyToManyField(Book)

#### 聚合函数
    聚合函数需要作为聚合方法的参数才能发挥作用。例如 Author.objects.annotate(Avg('age'))，该属性的键默认为字段名+双下划线+方法的组合，即age__avg, 若想自定义名称，则可写为：authors = Author.objects.annotate(avg_age=Avg('age')),此时该注解属性的名称为avg_age。通过author.avg_age访问。
    查询的连接可以尽可能的深，例如要查询作者写的书的价格区间。则:
        Author.objects.annotate(min_price=Min('books__price'),max_price=Max('books__price'))
    Avg：求平均值
    Min：求最小值
    Max：求最大值
    Sum：求和
    Count：统计数量
    

#### 注意
    应避免使用同一聚合函数组合多个聚合查询，否则可能会产生错误的结果。

In [ ]:
# 以下查询会导致authors__count和store__count得出错误的结果。 
q = Book.objects.annotate(Count('authors'), Count('store'))

#### annotate() 和 filter() 子句的顺序

In [ ]:
# 先进行注解，所有书的数量都被计算在内。
a, b = Publisher.objects.annotate(num_books=Count('book', distinct=True)).filter(book__rating__gt=3.0)

print(a, a.num_books)
(<Publisher: A>, 2)

print(b, b.num_books)   
(<Publisher: B>, 2)

# 先进行过滤，b中评分为3分以下的书并未参与注解的计算
a, b = Publisher.objects.filter(book__rating__gt=3.0).annotate(num_books=Count('book'))

print(a, a.num_books)
(<Publisher: A>, 2)

print(b, b.num_books) 
(<Publisher: B>, 1)

#### Q对象
    Q对象可用于‘或’的查询
    例如，filter可以进行并列过滤，即filter(name='xiaoming', age='23')。但是无法进行‘或‘的操作。

In [ ]:
# 相当于以‘Who’开头，并且满足date(2005, 5, 2)或者date(2005, 5, 6)中的一个。
Poll.objects.get(
    Q(question__startswith='Who'),
    Q(pub_date=date(2005, 5, 2)) | Q(pub_date=date(2005, 5, 6))
)

#### 通过~Q取反

In [ ]:
# 以‘Who’开头,并且不能为pub_date__year=2005
Poll.objects.get(
    Q(question__startswith='Who'), ~Q(pub_date__year=2005)
)


### F表达式
    F表达式是对数据库使用的一种优化，可以理解为其是定义好对数据库的一系列操作，并一次性执行完毕。
    
    一般情况下，倘若修改数据库中的某条数据，需要分两步进行：读取数据到内存中；重新赋值后再写入数据库。如此便执行了两次数据库操作。